# 🎯 Chest X-Ray Ensemble - 90%+ Target

**Baseline**: 82.3% | **Target**: 90%+

## Optimizations:
- torch.compile (20-40% speedup)
- 384px resolution option
- GPU cache + aggressive augmentation

## Config

In [ ]:
# ========== USER CONFIG ==========
USE_384 = True                  # True=Higher accuracy (384px), False=Faster (224px)
FAST_TEST = False               # True=5 min test, False=Full training

# ========== Setup ==========
import torch, os, shutil

if not torch.cuda.is_available():
    raise Exception("GPU required")

print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"Mode: {'384px (accuracy)' if USE_384 else '224px (speed)'}")
print(f"Test: {'Fast (5min)' if FAST_TEST else 'Full training'}")

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# Clone repo
%cd /content
if os.path.exists('nycu-CSIC30014-LAB3'):
    shutil.rmtree('nycu-CSIC30014-LAB3')
!git clone -q https://github.com/thc1006/nycu-CSIC30014-LAB3.git
%cd nycu-CSIC30014-LAB3

# Install
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install -q numpy pandas scikit-learn matplotlib tqdm pyyaml opencv-python kaggle scipy

# Kaggle API
from google.colab import files as colab_files
from pathlib import Path

uploaded = colab_files.upload()
if 'kaggle.json' not in uploaded:
    raise Exception("Upload kaggle.json")

kaggle_dir = Path.home() / '.kaggle'
kaggle_dir.mkdir(exist_ok=True)
kaggle_json = kaggle_dir / 'kaggle.json'
with open(kaggle_json, 'wb') as f:
    f.write(uploaded['kaggle.json'])
os.chmod(kaggle_json, 0o600)

print("✅ Setup complete")

## Data

In [ ]:
import zipfile
import subprocess
import pandas as pd
from tqdm.auto import tqdm

print("=" * 60)
print("STEP 2: DATA PREPARATION")
print("=" * 60)

# Download
print("\n[1/2] Downloading competition data...")
result = subprocess.run(
    ['kaggle', 'competitions', 'download', '-c', 'cxr-multi-label-classification'],
    capture_output=True, text=True
)

if result.returncode != 0:
    if '403' in result.stderr:
        print("❌ You need to join the competition first!")
        print("Visit: https://www.kaggle.com/competitions/cxr-multi-label-classification")
        raise Exception("Join competition")
    else:
        raise Exception(f"Download failed: {result.stderr}")

# Extract
for zip_file in [f for f in os.listdir('.') if f.endswith('.zip')]:
    with zipfile.ZipFile(zip_file, 'r') as zf:
        for file in tqdm(zf.namelist(), desc=f"Extracting {zip_file}", leave=False):
            zf.extract(file, '.')
    os.remove(zip_file)

print("✅ Data downloaded & extracted")

# Organize
print("\n[2/2] Organizing images...")

# Collect all images
all_images = {}
for search_dir in ['.', 'train_images', 'val_images', 'test_images']:
    if os.path.exists(search_dir):
        for fname in os.listdir(search_dir):
            if fname.endswith(('.jpg', '.jpeg', '.png')):
                if fname not in all_images:
                    all_images[fname] = os.path.join(search_dir, fname)

# Move CSVs to data/
os.makedirs('data', exist_ok=True)
for csv in ['train_data.csv', 'val_data.csv', 'test_data.csv']:
    if os.path.exists(csv) and not os.path.exists(f'data/{csv}'):
        shutil.move(csv, f'data/{csv}')

# Organize by split
splits = {
    'train': ('data/train_data.csv', 'train_images'),
    'val': ('data/val_data.csv', 'val_images'),
    'test': ('data/test_data.csv', 'test_images')
}

for split_name, (csv_path, target_dir) in splits.items():
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        needed_files = set(df['new_filename'].values)
        
        os.makedirs(target_dir, exist_ok=True)
        
        for fname in tqdm(needed_files, desc=f"{split_name.upper()}", leave=False):
            target_path = os.path.join(target_dir, fname)
            if not os.path.exists(target_path) and fname in all_images:
                source_path = all_images[fname]
                if os.path.abspath(source_path) != os.path.abspath(target_path):
                    try:
                        shutil.move(source_path, target_path)
                    except FileNotFoundError:
                        pass

# Verify
print("\nVerification:")
for split_name, (csv_path, target_dir) in splits.items():
    if os.path.exists(target_dir):
        count = len([f for f in os.listdir(target_dir) if f.endswith(('.jpg', '.jpeg', '.png'))])
        print(f"  {target_dir}: {count} images")

print("\n✅ Data organized")
print("=" * 60)

## Train 1

In [ ]:
if FAST_TEST:
    !python -m src.train_v2 --config configs/fast_test.yaml
else:
    suffix = "_384" if USE_384 else ""
    !python -m src.train_v2 --config configs/colab_convnext_tiny{suffix}.yaml
print("✅ Model 1 done")

## Train 2

In [ ]:
if not FAST_TEST:
    suffix = "_384" if USE_384 else ""
    !python -m src.train_v2 --config configs/colab_efficientnetv2_s{suffix}.yaml
    print("✅ Model 2 done")
else:
    print("⏭️ Skipped in FAST_TEST mode")

## Train 3

In [ ]:
if not FAST_TEST:
    suffix = "_384" if USE_384 else ""
    !python -m src.train_v2 --config configs/colab_regnetx_3.2gf{suffix}.yaml
    print("✅ Model 3 done")
else:
    print("⏭️ Skipped in FAST_TEST mode")

## TTA Prediction

In [ ]:
import yaml

if FAST_TEST:
    models = [('fast_test', 'configs/fast_test.yaml', 'outputs/fast_test/best.pt')]
else:
    suffix = "_384" if USE_384 else ""
    models = [
        ('convnext', f'configs/colab_convnext_tiny{suffix}.yaml', f'outputs/colab_convnext_tiny{suffix}/best.pt'),
        ('efficientnet', f'configs/colab_efficientnetv2_s{suffix}.yaml', f'outputs/colab_efficientnetv2_s{suffix}/best.pt'),
        ('regnet', f'configs/colab_regnetx_3.2gf{suffix}.yaml', f'outputs/colab_regnetx_3.2gf{suffix}/best.pt')
    ]

for name, config, ckpt in models:
    !python -m src.tta_predict --config {config} --ckpt {ckpt}
    
    with open(config, 'r') as f:
        cfg = yaml.safe_load(f)
    submission_path = cfg['out']['submission_path']
    
    if os.path.exists('submission_tta.csv'):
        shutil.move('submission_tta.csv', submission_path)
        
print("✅ TTA done")

## Ensemble

In [ ]:
import pandas as pd, numpy as np
from scipy.optimize import minimize

if FAST_TEST:
    print("⏭️ Skip ensemble in FAST_TEST mode")
else:
    suffix = "_384" if USE_384 else ""
    pred1 = pd.read_csv(f'data/submission_convnext_tiny{suffix}.csv')
    pred2 = pd.read_csv(f'data/submission_efficientnetv2_s{suffix}.csv')
    pred3 = pd.read_csv(f'data/submission_regnetx_3.2gf{suffix}.csv')
    
    prob_cols = ['normal', 'bacteria', 'virus', 'COVID-19']
    
    # Temperature scaling
    def temp_scale(logits, T):
        return logits / T
    
    def find_T(probs):
        avg_max = np.max(probs, axis=1).mean()
        return 1.5 if avg_max > 0.9 else (0.8 if avg_max < 0.7 else 1.0)
    
    T1 = find_T(pred1[prob_cols].values)
    T2 = find_T(pred2[prob_cols].values)
    T3 = find_T(pred3[prob_cols].values)
    
    scaled1 = np.exp(temp_scale(np.log(pred1[prob_cols].values + 1e-10), T1))
    scaled1 /= scaled1.sum(axis=1, keepdims=True)
    scaled2 = np.exp(temp_scale(np.log(pred2[prob_cols].values + 1e-10), T2))
    scaled2 /= scaled2.sum(axis=1, keepdims=True)
    scaled3 = np.exp(temp_scale(np.log(pred3[prob_cols].values + 1e-10), T3))
    scaled3 /= scaled3.sum(axis=1, keepdims=True)
    
    # Weighted ensemble
    weights = np.array([0.87, 0.88, 0.86])
    weights /= weights.sum()
    
    ensemble = pred1.copy()
    ensemble[prob_cols] = weights[0] * scaled1 + weights[1] * scaled2 + weights[2] * scaled3
    preds = ensemble[prob_cols].values.argmax(axis=1)
    ensemble[prob_cols] = np.eye(4)[preds]
    ensemble.to_csv('submission_ensemble.csv', index=False)
    
    print("✅ Ensemble done")
    for col in prob_cols:
        count = ensemble[col].sum()
        print(f"  {col}: {int(count)} ({count/len(ensemble)*100:.1f}%)")

## Download

In [ ]:
from google.colab import files as colab_files

if FAST_TEST:
    files = ['data/submission_fast_test.csv']
else:
    files = ['submission_ensemble.csv']

for f in files:
    if os.path.exists(f):
        colab_files.download(f)
        print(f"✅ {f}")
        
print("\n🎯 Expected: 88-92% (384px) or 85-88% (224px)")